<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
from glob import glob
files = glob("*_features.pkl")

with open(files[0], "rb") as f:
  test_file = pickle.load(f)


# features_with_labels beinhaltet fahrstil, schwung_labels, COM_to_ground, knee_angles_right, knee_angles_left, joint_angles, axis_angles, COM_angles
# fahrstil beinhaltet einen einzigen Wert, z.B. EKK -> Wert (einfach ein Wert)
# schwung_labels beinhalten für jeden Frame den Schwung, also Rechts oder Linksschwung (1 Linksschwung, 2 Rechtsschwung) Frame: Wert
# COM_to_ground beinhaltet für jeden Frame den Abstand zwischen COM und Boden Frame: Wert
# knee_angles_right und knee_angles_left beinhalten für jeden Frame die Kniewinkel (also Winkel zwischen Oberschenkel und Unterschenkel) -> Frame: Wert
# joint_angles gibt die Winkel zwischen den Segmenten (z.B. Oberarm, genannt shoulder to ellbow) und den 3 Achsen (x, y und z) -> Frame: { Achse: { Obarm links, Oberschenkel links, etc.}}
# axis_angles gibt die Winkel der Achsen (z.B. Schulterachse, Hüftachse, etc.) zu den 3 Achsen (x, y und z) -> Frame: { Achse: { Schulterachse, Hüftachse }}
# COM_angles gibt Winkel zwischen dem Vektor Boden zu COM und den 3 Achsen (x, y und z) -> Frame: {x-Winkel, y-Winkel, z-Winkel}
print("Fahrstil", test_file["fahrstil"])
print("Schwung Label (rechts oder links)", test_file["schwung_labels"])
print("COM zum Boden Abstand", test_file["COM_to_ground"])
print("Kniewinkel rechts", test_file["knee_angles_right"])
print("Winkel zwischen Segmenten und Achsen", test_file["joint_angles"])
print("Winkel zwischen Körperchsen (z.B. Schulterachse) und Achsen", test_file["axis_angles"])
print("Winkel zwischen Vektor Boden zum COM und den 3 Achsen", test_file["COM_angles"])

Fahrstil PGL
Schwung Label (rechts oder links) {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 2, 27: 2, 28: 2, 29: 2, 30: 2, 31: 2, 32: 2, 33: 2, 34: 2, 35: 2, 36: 2, 37: 2, 38: 2, 39: 2, 40: 2, 41: 2, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 2, 49: 2, 50: 2, 51: 2, 52: 2, 53: 2, 54: 2, 55: 2, 56: 2, 57: 2, 58: 2, 59: 2, 60: 2, 61: 2, 62: 2, 63: 2, 64: 2, 65: 2, 66: 2, 67: 2, 68: 2, 69: 2, 70: 2, 71: 2, 72: 2, 73: 2, 74: 2, 75: 2, 76: 2, 77: 2, 78: 2, 79: 2, 80: 2, 81: 2, 82: 2, 83: 2, 84: 2, 85: 2, 86: 2, 87: 2, 88: 2, 89: 2, 90: 2, 91: 2, 92: 2, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1, 100: 1, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 1, 111: 1, 112: 1, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 1, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 1, 130: 1, 131: 1, 132: 1

In [2]:
for key in test_file["schwung_labels"]:
  if (key == 0):
    print(test_file["COM_to_ground"][key])
    print(test_file["knee_angles_right"][key])
    print(test_file["knee_angles_left"][key])
    for axis in test_file["joint_angles"][key]:
      print(test_file["joint_angles"][key][axis].values())

    for axis in test_file["axis_angles"][key]:
      print(test_file["axis_angles"][key][axis].values())

    print(test_file["COM_angles"][key].values())

0.80812824
124.0553
119.690575
dict_values([np.float64(124.49609232237005), np.float64(66.05865620468326), np.float64(137.74924518202087), np.float64(136.75392303275672), np.float64(83.6482003157394), np.float64(88.35139584704844)])
dict_values([np.float64(50.800804493879845), np.float64(24.97834919525295), np.float64(48.09680413724409), np.float64(47.67723695676264), np.float64(23.605675700725936), np.float64(30.454300076242443)])
dict_values([np.float64(121.93532417352309), np.float64(83.29258276461466), np.float64(94.45530206948766), np.float64(82.71808475633605), np.float64(112.63465063033995), np.float64(120.40000313660266)])
dict_values([np.float64(120.58397084536828), np.float64(116.23578408349492), np.float64(109.22890600518701), np.float64(103.32165783818665)])
dict_values([np.float64(95.38163586701646), np.float64(106.68024532164426), np.float64(99.38054428289722), np.float64(94.25345219387277)])
dict_values([np.float64(31.15617578515749), np.float64(31.808086983683918), np.f

In [3]:
import numpy as np

# ich speichere die Werte in den Arrays, die y-Werte sind ein einfacher Array der Struktur [1, 2, 1, 1, 2], welcher für jeden Input Vektor den Output (Links- oder Rechtsschwung) ausgibt
# die data beinhaltet Unterarrays, die jeweils einen Wert für jedes Feature beinhalten, jeder Unterarray entspricht einem Frame
#  -> [[Frame 1 Feature 1, Frame 1 Feature 2, Frame 1, Feature 2], [Frame 2 Feature 1, Frame 2 Feature 2...], [ hier Werte dritter Frame alle Feature], ...]

data = []
labels = []
fahrstil_labels = []

MAX_LENGTH = 400

def augment_sequence(seq, noise_std=0.005, jitter_std=0.001, warp_strength=0.1, shift_max=10):
    seq = np.array(seq)

    # 0. Time Shift (zufällige Verschiebung nach vorn oder hinten)
    shift = np.random.randint(-shift_max, shift_max + 1)
    if shift > 0:
        seq = np.pad(seq, ((shift, 0), (0, 0)), mode='constant')[:len(seq)]
    elif shift < 0:
        seq = np.pad(seq, ((0, -shift), (0, 0)), mode='constant')[-shift:]
    # shift == 0 → bleibt unverändert

    # 1. Noise Injection
    seq += np.random.normal(0, noise_std, seq.shape)

    # 2. Jittering (feines zufälliges Zucken)
    seq += np.random.normal(0, jitter_std, seq.shape)

    # 3. Time Warping
    stretch_factor = 1 + np.random.uniform(-warp_strength, warp_strength)
    t_original = np.linspace(0, 1, len(seq))
    t_warped = np.linspace(0, 1, int(len(seq) * stretch_factor))
    warped = np.array([
        np.interp(t_original, np.clip(t_warped, 0, 1),
                  np.interp(t_warped, np.linspace(0, 1, len(seq)), seq[:, i]))
        for i in range(seq.shape[1])
    ]).T

    # Länge anpassen
    warped = warped[:len(seq)] if warped.shape[0] >= len(seq) else np.pad(warped, ((0, len(seq) - warped.shape[0]), (0, 0)))
    return warped



for file in files:
    with open(file, "rb") as f:
        features = pickle.load(f)

    sequence_data = []
    sequence_labels = []

    # Extrahiere Features für jeden Frame
    for frame_idx in features["schwung_labels"]:
        try:
            frame_features = [
                features["COM_to_ground"][frame_idx],
                features["knee_angles_right"][frame_idx],
                features["knee_angles_left"][frame_idx]
            ]

            # Füge joint_angles hinzu
            for axis in features["joint_angles"][frame_idx]:
                frame_features.extend(features["joint_angles"][frame_idx][axis].values())

            # Füge axis_angles hinzu
            for axis in features["axis_angles"][frame_idx]:
                frame_features.extend(features["axis_angles"][frame_idx][axis].values())

            # Füge COM_angles hinzu
            frame_features.extend(features["COM_angles"][frame_idx].values())

            sequence_data.append(frame_features)
            sequence_labels.append(features["schwung_labels"][frame_idx])

        except KeyError as e:
            print(f"Fehler in Datei {file}, Frame {frame_idx}: {str(e)}")
            break

    # Padding für zu kurze Sequenzen
    if len(sequence_data) < MAX_LENGTH:
        pad_length = MAX_LENGTH - len(sequence_data)
        pad_features = [0] * len(sequence_data[0]) if sequence_data else [0] * 27  # Annahme: 27 Features
        sequence_data += [pad_features] * pad_length
        sequence_labels += [0] * pad_length

    data.append(sequence_data)
    labels.append(sequence_labels)

    # Fahrstil vereinheitlichen
    vereinheitlicht = features["fahrstil"].replace("2", "")
    fahrstil_labels.append(vereinheitlicht)



# Konvertiere zu NumPy-Arrays
data = np.array(data, dtype=np.float32)
fahrstil_labels = np.array(fahrstil_labels)

print (data)

[[[  0.80812824 124.0553     119.690575   ...  81.72745    160.52966
   107.49755   ]
  [  0.8124922  123.10828    121.64312    ...  81.77593    159.63925
   108.49155   ]
  [  0.8080065  122.20404    118.43621    ...  81.720436   159.52374
   108.59067   ]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[  0.6964686   89.43506     83.2325     ... 104.54298    159.01958
   104.78749   ]
  [  0.6899556   87.76278     81.76267    ... 104.171555   159.7033
   104.225464  ]
  [  0.68269354  86.87317     80.28767    ... 103.36437    159.72722
   104.958824  ]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
  

In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from collections import Counter



# ------------------- Datenvorbereitung -------------------
# 1. Label Encoding für Fahrstile
le = LabelEncoder()
fahrstil_encoded = le.fit_transform(fahrstil_labels)

# 2. Daten-Normalisierung
scaler = StandardScaler()
original_shape = data.shape

# Skalierung durchführen
data_normalized = scaler.fit_transform(data.reshape(-1, data.shape[-1]))  # Flatten zu 2D
data_normalized = data_normalized.reshape(original_shape)  # Zurück zu 3D

with open("scaler_schwung.pkl", "wb") as f:
    pickle.dump(scaler, f)
with open("encoder_schwung.pkl", "wb") as f:
    pickle.dump(le, f)


# 3. Stratifizierte Train/Val/Test Split
X_train, X_temp, Y_train, Y_temp = train_test_split(
    data_normalized,
    fahrstil_encoded,
    test_size=0.3,
    stratify=fahrstil_encoded,
    random_state=42
)

X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp,
    Y_temp,
    test_size=0.5,
    stratify=Y_temp,
    random_state=42
)

# ✅ 👉 HIER Augmentierung NUR für das Trainingsset
X_train_aug = np.array([augment_sequence(x) for x in X_train])
Y_train_aug = Y_train.copy()

label_counts = Counter(Y_train)
median_count = np.median(list(label_counts.values()))

X_aug = []
Y_aug = []

for x, y in zip(X_train, Y_train):

      for _ in range(3):  # 3x Augmentierung pro Sample der Minderheitsklasse
        x_aug = augment_sequence(x)
        X_aug.append(x_aug)
        Y_aug.append(y)


# 4. Kombiniere Original + augmentierte Daten
X_train_combined = np.concatenate([X_train, np.array(X_aug)])
Y_train_combined = np.concatenate([Y_train, np.array(Y_aug)])


# 5. Konvertierung zu PyTorch Tensoren (KORRIGIERT)
X_train = torch.tensor(X_train_combined, dtype=torch.float32)
Y_train = torch.tensor(Y_train_combined, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)


# ------------------- DataLoader -------------------
batch_size = 16
train_dataset = TensorDataset(X_train, Y_train)
val_dataset = TensorDataset(X_val, Y_val)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [5]:
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

# ------------------- Modellarchitektur -------------------
class SkiSwingLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.3
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 wegen bidirectional

    def forward(self, x):
        output, (hidden, _) = self.lstm(x)
        # Kombiniere die letzten Hidden States beider Richtungen
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        return self.fc(self.dropout(hidden))

# ------------------- Training Setup -------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SkiSwingLSTM(
    input_size=data.shape[-1],  # Anzahl Features pro Frame
    hidden_size=256,
    num_classes=len(le.classes_)
).to(device)

# Klassen-Gewichte
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(Y_train.numpy()),
    y=Y_train.numpy()
)
weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# ------------------- Training -------------------
best_val_accuracy = 0
patience = 50
patience_counter = 0

for epoch in range(300):
    # Trainingsschleife
    model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Validierungsschleife
    model.eval()
    val_preds = []
    val_true = []
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            preds = model(batch_x.to(device)).argmax(dim=1).cpu()
            val_preds.extend(preds.numpy())
            val_true.extend(batch_y.numpy())

    val_accuracy = np.mean(np.array(val_preds) == np.array(val_true))
    print(f"Epoch {epoch+1:3d} | Val Accuracy: {val_accuracy:.4f}")

    # Early Stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early Stopping nach {epoch+1} Epochen")
            break

# ------------------- Evaluation -------------------
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

all_preds = []
all_true = []
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        preds = model(batch_x.to(device)).argmax(dim=1).cpu()
        all_preds.extend(preds.numpy())
        all_true.extend(batch_y.numpy())

print("\nKlassifikationsbericht:")
print(classification_report(
    all_true,
    all_preds,
    labels=range(len(le.classes_)),
    target_names=le.classes_,
    zero_division=0
))

Epoch   1 | Val Accuracy: 0.2736
Epoch   2 | Val Accuracy: 0.2453
Epoch   3 | Val Accuracy: 0.2736
Epoch   4 | Val Accuracy: 0.2642
Epoch   5 | Val Accuracy: 0.5660
Epoch   6 | Val Accuracy: 0.6792
Epoch   7 | Val Accuracy: 0.7736
Epoch   8 | Val Accuracy: 0.7830
Epoch   9 | Val Accuracy: 0.7547
Epoch  10 | Val Accuracy: 0.6792
Epoch  11 | Val Accuracy: 0.5755
Epoch  12 | Val Accuracy: 0.7170
Epoch  13 | Val Accuracy: 0.7925
Epoch  14 | Val Accuracy: 0.6415
Epoch  15 | Val Accuracy: 0.7642
Epoch  16 | Val Accuracy: 0.8113
Epoch  17 | Val Accuracy: 0.8396
Epoch  18 | Val Accuracy: 0.7925
Epoch  19 | Val Accuracy: 0.8585
Epoch  20 | Val Accuracy: 0.7830
Epoch  21 | Val Accuracy: 0.8868
Epoch  22 | Val Accuracy: 0.8679
Epoch  23 | Val Accuracy: 0.8774
Epoch  24 | Val Accuracy: 0.8491
Epoch  25 | Val Accuracy: 0.8585
Epoch  26 | Val Accuracy: 0.8585
Epoch  27 | Val Accuracy: 0.8302
Epoch  28 | Val Accuracy: 0.8019
Epoch  29 | Val Accuracy: 0.8019
Epoch  30 | Val Accuracy: 0.8302
Epoch  31 

In [6]:
print(le.classes_)


['CL' 'EKK' 'PD' 'PDK' 'PDL' 'PGK' 'PGL' 'PS']


In [7]:
torch.save(model, "ski_schwung_classifier.pt")

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_temp, Y_train, Y_temp = train_test_split(
    data,
    labels,
    test_size=0.3,
    random_state=42,
    shuffle=True
)

X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp,
    Y_temp,
    test_size=0.5,
    random_state=42,
    shuffle=True
)


In [9]:
data = []
labels = []
fahrstil_labels = []

# Fahrstile vorher encodieren
fahrstile_raw = [pickle.load(open(f, "rb"))["fahrstil"] for f in files]
le = LabelEncoder()
le.fit(fahrstile_raw)

for file in files:
    with open(file, "rb") as f:
        features = pickle.load(f)

    sequence_data = []
    sequence_labels = []

    fahrstil_encoded = le.transform([features["fahrstil"]])[0]  # als Zahl

    for frame_idx in features["schwung_labels"]:
        try:
            frame_features = [
                features["COM_to_ground"][frame_idx],
                features["knee_angles_right"][frame_idx],
                features["knee_angles_left"][frame_idx]
            ]

            for axis in features["joint_angles"][frame_idx]:
                frame_features.extend(features["joint_angles"][frame_idx][axis].values())

            for axis in features["axis_angles"][frame_idx]:
                frame_features.extend(features["axis_angles"][frame_idx][axis].values())

            frame_features.extend(features["COM_angles"][frame_idx].values())

            frame_features.append(fahrstil_encoded)  # Fahrstil konstant pro Frame

            sequence_data.append(frame_features)
            sequence_labels.append(features["schwung_labels"][frame_idx])

        except KeyError as e:
            print(f"Fehler in Datei {file}, Frame {frame_idx}: {str(e)}")
            break

    if len(sequence_data) < MAX_LENGTH:
        pad_len = MAX_LENGTH - len(sequence_data)
        pad_feat = [0] * len(sequence_data[0]) if sequence_data else [0] * 28
        sequence_data += [pad_feat] * pad_len
        sequence_labels += [0] * pad_len

    data.append(sequence_data)
    labels.append(sequence_labels)
    fahrstil_labels.append(fahrstil_encoded)


In [10]:
data = np.array(data, dtype=np.float32)
labels = np.array(labels, dtype=np.int64)

X_train, X_temp, Y_train, Y_temp = train_test_split(data, labels, test_size=0.3, random_state=42, shuffle=True)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42, shuffle=True)

X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, Y_val), batch_size=16)
test_loader = DataLoader(TensorDataset(X_val, Y_val), batch_size=16)



In [11]:
class FrameWiseLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True, bidirectional=True, dropout=0.3)
    self.dropout = nn.Dropout(0.3)
    self.fc = nn.Linear(hidden_size * 2, num_classes)

  def forward(self, x):
    lstm_out, _ = self.lstm(x)
    return self.fc(self.dropout(lstm_out))

device = torch.device("cuda" if torch.cuda.is_available() else "cpi")
model = FrameWiseLSTM(input_size=data.shape[-1], hidden_size=128, num_classes=3).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_val_accuracy = 0
patience = 30
patience_counter = 0

for epoch in range(100):
  model.train()
  for batch_x, batch_y in train_loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    optimizer.zero_grad()
    output = model(batch_x)
    output = output.permute(0, 2, 1)
    loss = loss_fn(output, batch_y)
    loss.backward()
    optimizer.step()

  model.eval()
  val_preds = []
  val_true = []
  with torch.no_grad():
    for batch_x, batch_y in val_loader:
      batch_x = batch_x.to(device)
      preds = model(batch_x).argmax(dim=-1).cpu()
      val_preds.append(preds)
      val_true.append(batch_y)




  val_preds = torch.cat(val_preds).flatten().numpy()
  val_true = torch.cat(val_true).flatten().numpy()

  # ❗ Padding rausfiltern (0 = kein echtes Label)
  mask = val_true != 0
  val_accuracy = np.mean(val_preds[mask] == val_true[mask])

  print(f"Epoch {epoch+1:3d} | Val Accuracy: {val_accuracy:.4f}")


  if val_accuracy > best_val_accuracy:
    best_val_accuracy = val_accuracy
    patience_counter = 0
    torch.save(model.state_dict(), "best_model_framewise.pth")
  else:
    patience_counter += 1
    if patience_counter >= patience:
      print(f"Early Stopping at Epoch {epoch + 1}")
      break;


model.load_state_dict(torch.load("best_model_framewise.pth"))
model.eval()
all_preds = []
all_true = []

with torch.no_grad():
  for batch_x, batch_y in test_loader:
    batch_x = batch_x.to(device)
    preds = model(batch_x).argmax(dim=-1).cpu()
    all_preds.append(preds)
    all_true.append(batch_y)

all_preds = torch.cat(all_preds).flatten().numpy()
all_true = torch.cat(all_true).flatten().numpy()

# ❗ Nur echte Labels vergleichen
mask = all_true != 0
print("\nKlassifikationsbericht:")
print(classification_report(all_true[mask], all_preds[mask], digits=3, zero_division=0))

Epoch   1 | Val Accuracy: 0.9036
Epoch   2 | Val Accuracy: 0.9149
Epoch   3 | Val Accuracy: 0.9239
Epoch   4 | Val Accuracy: 0.9248
Epoch   5 | Val Accuracy: 0.9225
Epoch   6 | Val Accuracy: 0.9132
Epoch   7 | Val Accuracy: 0.9162
Epoch   8 | Val Accuracy: 0.9300
Epoch   9 | Val Accuracy: 0.9301
Epoch  10 | Val Accuracy: 0.9386
Epoch  11 | Val Accuracy: 0.9233
Epoch  12 | Val Accuracy: 0.9320
Epoch  13 | Val Accuracy: 0.9324
Epoch  14 | Val Accuracy: 0.9350
Epoch  15 | Val Accuracy: 0.9422
Epoch  16 | Val Accuracy: 0.9370
Epoch  17 | Val Accuracy: 0.9249
Epoch  18 | Val Accuracy: 0.9401
Epoch  19 | Val Accuracy: 0.9427
Epoch  20 | Val Accuracy: 0.9328
Epoch  21 | Val Accuracy: 0.9306
Epoch  22 | Val Accuracy: 0.9413
Epoch  23 | Val Accuracy: 0.9458
Epoch  24 | Val Accuracy: 0.9408
Epoch  25 | Val Accuracy: 0.9415
Epoch  26 | Val Accuracy: 0.9451
Epoch  27 | Val Accuracy: 0.9351
Epoch  28 | Val Accuracy: 0.9285
Epoch  29 | Val Accuracy: 0.9154
Epoch  30 | Val Accuracy: 0.9327
Epoch  31 

In [12]:
torch.save(model, "left_right_classifier.pt")